In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
from core.utils import *
%matplotlib inline 

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /Users/dungnd/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-6-18 Python-3.10.9 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp2/weights/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/dungnd/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-6-18 Python-3.10.9 torch-2.0.0 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
def pre_processing_img(crop_img):
    # grayscale region within bounding box
    gray = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)

    # resize image to three times as large as original for better readability
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)

    # perform gaussian blur to smoothen image
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # threshold the image using Otsus method to preprocess for tesseract
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    # create rectangular kernel for dilation
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    # apply dilation to make regions more clear
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    # Invert the black and white background image
    inv_thresh = cv2.bitwise_not(thresh)
    inv_thresh = cv2.bitwise_not(inv_thresh)

    # Perform bitwise AND operation between the grayscale image and inverted background
    result = cv2.bitwise_and(dilation, dilation, mask=inv_thresh)
    # perform another blur on character region
    result = cv2.medianBlur(result, 5)

    return result

In [5]:
# img_path = os.path.join('data', 'images', 'CarLongPlate137.jpg')
img_path = os.path.join('test_img', 'test_9.jpeg')
# img_path = 'https://www.roadpol.eu/images/2022/SEP/Russian.jpg'
# img_path = 'https://image.vietnamnews.vn/uploadvnnews/Article/2017/6/15/5b3929415d1_wize16423115PM.jpg'
# img_path = 'https://media.urbanistnetwork.com/saigoneer/article-images/legacy/lB85cVQb.jpg'

In [6]:
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img)
results.print()

image 1/1: 4032x3024 1 plate
Speed: 18.2ms pre-process, 206.8ms inference, 20.4ms NMS per image at shape (1, 3, 640, 480)


In [7]:
results.pandas().xywh[0]

,xcenter,ycenter,width,height,confidence,class,name
0,1483.689575,2212.386475,886.713379,688.322144,0.964241,0,plate


In [8]:
# Save cropped image
delete_all_cropped()
lp_list = results.pandas().xywh[0].values.tolist()
for idx, lp in enumerate(lp_list):
    x,y,w,h =  lp[0], lp[1], lp[2], lp[3]
    crop_img = img[int(y-h//2-5):int(y+h//2+5) , int(x-w//2-5):int(x+w//2+5)]
    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
#     result = pre_processing_img(crop_img)
    result = crop_img
    cv2.imwrite("crop_images/plate_" + str(idx+1) + ".jpeg", result)
    print("Save cropped image - crop_images/plate_" + str(idx+1) + ".jpeg")

Save cropped image - crop_images/plate_1.jpeg


In [9]:
plt.imshow(np.squeeze(results.render()))
plt.show()